In [6]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

In [9]:
class PleuralEffusionDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.image_dir, img_name)
        label_path = os.path.join(self.label_dir, img_name.replace('.jpg', '.txt'))

        image = Image.open(img_path).convert("RGB")

        # Lê a primeira classe do arquivo de label
        with open(label_path, 'r') as f:
            line = f.readline()
            label = int(line.split()[0])  # classe 0 ou 1

        if self.transform:
            image = self.transform(image)

        return image, label

In [14]:
base_path = 'drive/MyDrive/Arquivos/plueral effusion'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = PleuralEffusionDataset(
    os.path.join(base_path, 'train/images'),
    os.path.join(base_path, 'train/labels'),
    transform)

valid_dataset = PleuralEffusionDataset(
    os.path.join(base_path, 'valid/images'),
    os.path.join(base_path, 'valid/labels'),
    transform)

test_dataset = PleuralEffusionDataset(
    os.path.join(base_path, 'test/images'),
    os.path.join(base_path, 'test/labels'),
    transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8)
test_loader  = DataLoader(test_dataset, batch_size=8)

In [15]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 16 x 112 x 112
        x = self.pool(F.relu(self.conv2(x)))  # 32 x 56 x 56
        x = x.view(-1, 32 * 56 * 56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}")


Epoch [1/10], Loss: 20.3411
Epoch [2/10], Loss: 12.3402
Epoch [3/10], Loss: 12.1438
Epoch [4/10], Loss: 11.3730
Epoch [5/10], Loss: 10.2123
Epoch [6/10], Loss: 8.9795
Epoch [7/10], Loss: 7.1483
Epoch [8/10], Loss: 6.6854
Epoch [9/10], Loss: 4.8113
Epoch [10/10], Loss: 3.3331


In [17]:
def evaluate(loader, name="Set"):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    print(f"{name} Accuracy: {correct / total:.2%}")

evaluate(valid_loader, "Validation")
evaluate(test_loader, "Test")


Validation Accuracy: 62.50%
Test Accuracy: 75.00%
